In [ ]:
import re
from lexical_benchmark.datasets.human.childes_clean import TagCleaner

pattern = re.compile(r"\b[^ ]+@o\b")
cleaner = TagCleaner(clean_pattern="@o", match_pattern=pattern, )

In [22]:
import re
examples = [
    "Single word: Hello@o",
    "Multiple words: This is a word@o",
    "Punctuation: I love@o! It's great.",
    "Numbers: 123@o is a weird number.",
    "Capitalization: World@o is a big place.",
    "Special characters: $@o is a symbol.",
    "Multiple instances: Hello@o, world@o!",
    "Beginning of line: @o is the first word.",
    "End of line: The last word is @o.",
    "Mixed case: My@o favorite animal is a cat@o."
]
pattern = re.compile(r"\b[^ ]+@o\b")
for i, e in enumerate(examples):
    res = pattern.sub(" ", e)
    print(f"{i}: {res}")

0: Single word:  
1: Multiple words: This is a  
2: Punctuation: I  ! It's great.
3: Numbers:   is a weird number.
4: Capitalization:   is a big place.
5: Special characters: $@o is a symbol.
6: Multiple instances:  ,  !
7: Beginning of line: @o is the first word.
8: End of line: The last word is @o.
9: Mixed case:   favorite animal is a  .
